In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
# column headers
columns = ["name", "type", "A", "B", "Rh"]

# binary list csv for categorical testing
bnry_data = pd.read_csv("bnry_blood.csv", names=columns) 

print(bnry_data)

        name type  A  B  Rh
0   hu43860C   O+  0  0   1
1   huC30901   O-  0  0   0
2   hu9385BA   A+  1  0   1
3   hu04FD18   O+  0  0   1
4   huAE6220   B+  0  1   1
5   hu604D39  AB+  1  1   1
6   hu6E4515   O+  0  0   1
7   hu2FEC01   A+  1  0   1
8   hu2DBF2D   O-  0  0   0
9   hu72A81D   A+  1  0   1
10  hu2D6140   O+  0  0   1
11  huA0E089   A+  1  0   1
12  hu8229AE   O+  0  0   1
13  huCA017E   A+  1  0   1
14  hu38168C   O+  0  0   1
15  huD81F3D   A+  1  0   1
16  huD37D14   O+  0  0   1
17  huBAAC98   O+  0  0   1
18  huEC6EEC   O+  0  0   1
19  huF5E0B6   A+  1  0   1
20  huAEADC0   A-  1  0   0
21  hu241DEA   A-  1  0   0
22  hu025CEA   O-  0  0   0
23  hu050E9C   O-  0  0   0
24  hu955EE1   O+  0  0   1
25  huA4E2CF   B+  0  1   1
26  huBA30D4   O+  0  0   1
27  hu05FD49   A+  1  0   1
28  huC14AE1   A-  1  0   0
29  hu2C1D94   A+  1  0   1
..       ...  ... .. ..  ..
55  hu040C0A   A-  1  0   0
56  hu04F220   B+  0  1   1
57  huDBF9DD   B+  0  1   1
58  hu33E2D9   O+  0

In [27]:
# load the formatted pgp numpy array
pgp_filtered = np.load("pgp_filtered_blood.pkl")

# Scale the data
from sklearn import preprocessing
pgp_filtered = preprocessing.scale(pgp_filtered.astype('double')) # scale the data

In [28]:
print(pgp_filtered.shape)

(85, 2564734)


In [29]:
#bnry_data = np.asarray(bnry_data)

# insert row condition here to test
condition = "A"

X = pgp_filtered
y = bnry_data[condition]

# print the shapes of X and y
print(X.shape)

(85, 2564734)


In [11]:
###############################################################################
#                                                                             #
#           Machine Learning Classifiers and  Accuracy Metrics                #
#                                                                             #
###############################################################################

In [34]:
#################################### Linear SVC Classifier  ######################################################

from sklearn import svm
from sklearn.svm import LinearSVC 

# set the c score here
cv = 0.01 # 0.05:1.0, 
#.001

svc_test = LinearSVC(penalty='l1', class_weight='balanced', C=cv, dual=False)


In [35]:
classifier = svm.LinearSVC(penalty='l1', dual=False, C=0.01)
svc = classifier.fit(X, y)

In [36]:
n = 10
scores = cross_val_score(svc_test, X, y, cv=n)

print(scores)

[ 0.6         0.6         0.55555556  0.625       0.625       0.625       0.625
  0.625       0.625       0.625     ]


In [10]:
##############################    KFold cross validation    ##################################

from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import accuracy_score, confusion_matrix
 
    
# Enter Number of Splits
splits =10
kf = KFold(n_splits=splits, random_state=None, shuffle=False)
    
scoresk = []
predictedk = []
testedk = []
    
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    svc_test.fit(X_train, y_train.ravel())
    y_pred = svc_test.predict(X_test)
    scoresk.append(accuracy_score(y_test, y_pred))
    
    predictedk.append(y_pred)
    testedk.append(y_test)
    
    
print(np.mean(np.asarray(scoresk)))

0.748611111111


In [102]:
############################################# Train Test Split  ######################################################

from sklearn.model_selection import cross_val_score, train_test_split, LeaveOneOut, KFold
from sklearn.metrics import accuracy_score, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2) #, random_state=4)
    
svc_test.fit(X_train, y_train.ravel())
y_pred = svc_test.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.692307692308


In [98]:
print(list(map(list, predictedk)))

print(list(map(list, testedk)))

[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 1, 0, 0], [0, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0]]


In [ ]:
###############################################################################
#                                                                             #
#                         Look Up Associated Genes                            #
#                                                                             #
###############################################################################

In [107]:
nonzeroes = np.nonzero(svc_test.coef_[0])[0]
coefs = zip(nonzeroes, svc_test.coef_[0][nonzeroes])
print(svc_test)

LinearSVC(C=0.09, class_weight='balanced', dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=None, tol=0.0001,
     verbose=0)


In [108]:
# sort the coefficients by their value, instead of index

coefs = sorted(coefs, key = lambda x: x[1], reverse=True)

coef_lst = []

for coef in coefs:
    print("#",coef, cv)
    coef_lst.append(coef)

# (2441217, 0.30078201055425646) 0.09
# (717404, 0.14091644581447238) 0.09
# (1365984, 0.11448739751186601) 0.09
# (957915, 0.047686298330681012) 0.09
# (2497657, 0.033719953880570815) 0.09
# (1469872, 0.026195461421963922) 0.09
# (495438, 0.024948451208477548) 0.09
# (2497720, 0.021087424522296647) 0.09
# (1997180, 0.016477051676918937) 0.09
# (1422739, 0.015189784369833441) 0.09
# (1479431, 0.0052165529095409315) 0.09
# (1310568, 0.0045246332019278843) 0.09
# (799964, -0.073764400268420585) 0.09
# (856102, -0.1794502101934444) 0.09


In [ ]:
# Results

# Experiment with RH Factor A
# nothing

# Experiment with B
# (2108511, 0.32772278909994074) 0.05 || hg19:chr20:0302	142528	155267373	15	16
# (1713737, 0.31411426306215895) 0.05 || hg19:chr13:0260	280816	126550875	15	16
# (2441217, 0.12062420103056076) 0.03
# (2441217, 0.30078201055425646) 0.09
# (717404, 0.14091644581447238) 0.09


# 200879, 0.22380127817290715  c = 0.07  - experiment || hg19:chr1:003e 313504 13078864

# 2082638, 0.19800320891630482  c = 0.09  - experiment || hg19:chr20:02f9 308544 15364156

# (1424406, 0.11788779754186479) 0.04
# (207585, 0.092608897347778468) 0.04

In [168]:
# save just the coefficient values
firstCoefs = [coef[0] for coef in coefs]
indices = np.asarray(firstCoefs)

# dump the coefficients for tiling analysis
indices.dump("coefs.pkl")